In [3]:
import pandas as pd
import numpy as np
from pylab import mpl, plt

#read the data
raw=pd.read_csv('./OneMonthData.csv', index_col=0, parse_dates=True).dropna()
data=pd.DataFrame(raw[['time','close']][0:5000])
data['returns']=np.log(data['close']/data['close'].shift(1))
#engineer the features
window=20
data['vol']=data['returns'].rolling(window).std()
data['mom']=data['returns'].rolling(window).mean()
data['sma']=data['close'].rolling(window).mean()
data['min']=data['close'].rolling(window).min()
data['max']=data['close'].rolling(window).max()
data.dropna(inplace=True)

#create the feature vectors
lags=5
features=['returns', 'vol', 'mom', 'sma', 'min', 'max']
dataLags=np.lib.stride_tricks.sliding_window_view(data[features], window_shape=lags, axis=0)[:-1,:].reshape((-1, len(features)*lags))
data['direction']=np.where(data['returns']>0, 1, -1)
data.dropna(inplace=True)

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

n_estimators=15
random_state=100
max_depth=3
min_samples_leaf=15
subsample=0.33

#create the model
tree=DecisionTreeClassifier(random_state=random_state, max_depth=max_depth, min_samples_leaf=min_samples_leaf)
model=AdaBoostClassifier(base_estimator=tree, n_estimators=n_estimators, random_state=random_state)

#normalise the data
mu=np.mean(dataLags, axis=0)
std=np.std(dataLags, axis=0)
normalisedData=(dataLags-mu)/std

#fir the normalised training data o the model created
model.fit(normalisedData, data['direction'][lags:])


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3,
                                                         min_samples_leaf=15,
                                                         random_state=100),
                   n_estimators=15, random_state=100)

In [4]:
import numpy as np
import pandas as pd
from pylab import mpl, plt
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'

'''
Base class for the event-based testing of the trading strategies

Attributes
------------------------------
column : column to be used for trading
start  : starting date for the collection of data for trading
end    : ending date for the collection of data for trading
amount : amount to be traded 
ftc    : fixed transaction costs
ptc    : proportional transaction costs

Functions
-----------------------------------
get_data()         : retrieves the data for implementing the trading strategy
plot_data()        : plots the retrieved data
get_price()        : returns the price for the corresponding bar
print_balance()    : prints the current balance left
print_net_wealth() : prints the current net wealth
place_buy_order()  : places an order to buy foreign exchange
place_sell_order() : places an order to sell foreign exchange
close_out()        : closes the implementation of the trade
'''
class BacktestBase(object):
    def __init__(self, amount, ftc=0.0, ptc=0.0, verbose=True):
        self.initialAmount=amount    #stores the inital ammount allotted for trading (remains constant)
        self.amount=amount           #trading cash balance (running balance)
        self.ftc=ftc                 #fixed transaction costs per trade
        self.ptc=ptc                 #propertional transaction costs per trade
        self.units=0                 #number of units traded
        self.position=0              #initial position is set to neutral
        self.trades=0                #no trades are executed initially
        self.verbose=verbose         #prints detailed summary in terminal

    def set_prices(self, price):
        '''
        Keep track of prices to track the performance
        '''
        self.entry_price=price
        self.min_price=price
        self.max_price=price
        
    def print_balance(self, time):
        '''
        For the requested bar, prints the available balance
        '''
        print(f'{time} | current balance {self.amount:.2f}')

    def print_net_wealth(self, time, price):
        '''
        For the requested bar, prints the net wealth
        '''
        net_wealth=self.units*price+self.amount
        print(f'{time} | current net wealth {net_wealth:.2f}')
        
    def place_buy_order(self, time, price, units=None, amount=None):
        '''
        Place an order to buy foreign exchange for the passed amount or the passed number of units at the corresponding bar
        '''
        if units is None:               #if the number of unites are not given
            units=int(amount/price)     #number of units are assumed to be all units
        self.amount-=(units*price)*(1+self.ptc)+self.ftc   #update the current balance
        self.units+=units               #the number of units are updated
        self.trades+=1                  #the number of trades is increased by one
        self.set_prices(price)          #the prices are updated
        if self.verbose==True:
            print(f'{time} | buying {units} units at {price:.3f}')
            self.print_balance(time)    #current banace is printed
            self.print_net_wealth(time, price)             #current wealth is printed
            
    def place_sell_order(self, time, price, units=None, amount=None):
        '''
        Place an order to sell foreign exchange for the passed amount or the passed number of units at the corresponding bar
        '''
        if units is None:               #if the number of unites are not given
            units=int(amount/price)     #number of units are assumed to be all units
        self.amount+=(units*price)*(1-self.ptc)-self.ftc    #update the current balance
        self.units-=units               #the number of units are updated
        self.trades+=1                  #the number of trades is increased by one
        self.set_prices(price)          #the prices are updated
        if self.verbose==True:
            print(f'{time} | selling {units} units at {price:.3f}')
            self.print_balance(time)    #current banace is printed
            self.print_net_wealth(time, price)              #current wealth is printed
            
    def close_out(self, time, price):
        '''
        Closes out any remaining open position at the corresponding bar 
        '''
        #self.amount+=self.units*price
        if self.units>0:
            self.place_sell_order(time, price, units=self.units)
        print(50*'_')
        print(f'{time} | --- CLOSING OUT ---')
        if self.verbose==True:
            print(f'{time} | remianing {self.units} units at {price:.3f}')
            print(50*'-')
        print('Final balance [$] {:.3f}'.format(self.amount))    #print final balance as the cash balance plus the price of assets
        perf = ((self.amount-self.initialAmount)/self.initialAmount*100)
        print('Net Performance [%] {:.3f}'.format(perf))         #calculate the net performance
        print('Trades Executed [#] {:.3f}'.format(self.trades))
        print(50*'_')

In [17]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

#create three real time streaming sub-plots
frame=make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("EUR_USD EXC Rate", "Returns"))
frame.append_trace(go.Scatter(name='EXCHANGE RATE'), row=1, col=1)
frame.append_trace(go.Scatter(name='returns'), row=2, col=1)
frame.append_trace(go.Scatter(name='momentum'), row=2, col=1)
frame.update_layout(height=800)

#create an updatable Figure Widget
fig=go.FigureWidget(frame)
fig


FigureWidget({
    'data': [{'name': 'EXCHANGE RATE',
              'type': 'scatter',
              'uid': 'c…

In [18]:
#run this cell along with tick_server.py in a separate terminal instance
import numpy as np
import pandas as pd
import zmq                           #import the ZeroMQ Python wrapper library
import datetime

class BacktestLongOnly(BacktestBase):
    
    def get_prediction(self, data):
        '''
        Function to return the prediction for the returns when passes the required data
        '''
        calc_data=pd.DataFrame(data[-(window+lags+1):])                  #extract the required values
        calc_data['returns']=np.log(calc_data/calc_data.shift(1))        #engineer the features
        calc_data['vol']=calc_data['returns'].rolling(window).std()
        calc_data['mom']=calc_data['returns'].rolling(window).mean()
        calc_data['sma']=calc_data['close'].rolling(window).mean()
        calc_data['min']=calc_data['close'].rolling(window).min()
        calc_data['max']=calc_data['close'].rolling(window).max()
        #update the Figure Widget
        fig.data[0].x=self.times
        fig.data[0].y=calc_data['close']
        fig.data[1].x=self.times
        fig.data[1].y=calc_data['returns']
        fig.data[2].x=self.times
        fig.data[2].y=calc_data['mom']
        self.times.pop(0)
        calc_data.dropna(inplace=True)
        features=['returns', 'vol', 'mom', 'sma', 'min', 'max']           #prepare the feature vectors
        feature_vector=np.lib.stride_tricks.sliding_window_view(calc_data[features], window_shape=lags, axis=0)[:-1,:].reshape((-1, len(features)*lags))
        feature_vector=(feature_vector-mu)/std                            #normalise the data
        action=model.predict(feature_vector)                              #predict the returns for the next tick value
        #share the Figure Widget accordingly
        if action==1:
            fig.add_vrect(
                x0=self.times[-1], x1=self.times[-2],
                fillcolor="green", opacity=0.5,
                layer="below", line_width=0,
            )
        if action==-1:
            fig.add_vrect(
                x0=self.times[-1], x1=self.times[-2],
                fillcolor="red", opacity=0.5,
                layer="below", line_width=0,
            )
        return action
        
    def run_testing(self, window=20, lags=5):
        
        context = zmq.Context()              #create an instance of the Context object
        socket = context.socket(zmq.SUB)     #a subscriber type socket is created
        socket.connect('tcp://0.0.0.0:5555') #connect to the IP address and port of the server
        socket.setsockopt_string(zmq.SUBSCRIBE, 'close')   #subscribe to the required channel 
        
        text=f'\n\nRunning Decision Tree based strategy (online) | lags={lags}'
        text+=f'\nfixed costs {self.ftc} | '
        text+=f'proportional costs {self.ptc}'
        print(text)                     #print the parameters
        print('='*55)
        self.units=0
        self.position=0                 #initially the position is neutral
        self.trades=0                   #no trades have been exeecuted yet
        self.amount=self.initialAmount  #reset initial capital
        self.sl=0.0001
        self.tsl=0.0001
        self.tp=0.0010
        data=pd.DataFrame()             #create an empty pandas dataframe
        tickval=0
        self.times=list()
        
        while tickval<7000:                 #loop to receive the data
            received=socket.recv_string()   #receive the datafrom the defined socket
            print(received)
            t=datetime.datetime.now()
            self.times.append(t)
            sym, val=received.split()
            val=float(val)
            data=data.append(pd.DataFrame({sym: val}, index=[t]))
            if len(data)>window+lags:
                if self.trades==0:
                    print(50*'_')
                    print(f'{tickval} | --- START TESING ---')
                    print(50*'-') 
                if self.sl!=0 and self.position!=0:
                    change=(val-self.entry_price)/self.entry_price
                    if self.position==1 and change<-self.sl:
                        print(50*'-')
                        print('--- STOP LOSS (LONG | -{self.sl}) ---')
                        self.place_buy_order(time=t, price=val, units=self.units)
                        self.position=-1
                    elif self.position==-1 and change>self.sl:
                        print(50*'-')
                        print('--- STOP LOSS (SHORT | -{self.sl}) ---')
                        self.place_sell_order(time=t, price=val, units=self.units)
                        self.position=1
                if self.tsl!=0 and self.position!=0:
                    self.max_price=max(self.max_price, val)
                    self.min_price=min(self.min_price, val)
                    if self.position==1 and (val-self.max_price)/self.entry_price<-self.tsl:
                        print(50*'-')
                        print('--- TRAILING STOP LOSS (LONG | -{self.tsl}) ---')
                        self.place_buy_order(time=t, price=val, units=self.units)
                        self.position=-1
                    elif self.position==-1 and (self.min_price-val)/self.entry_price<-self.tsl:
                        print(50*'-')
                        print('--- TRAILING STOP LOSS (SHORT | -{self.tsl}) ---')
                        self.place_sell_order(time=t, price=val, units=self.units)
                        self.position=1
                if self.tp!=0 and self.position!=0:
                    change=(val-self.entry_price)/self.entry_price
                    if self.position==1 and change>self.tp:
                        print(50*'-')
                        print('--- TAKE PROFIT (LONG | -{self.sl}) ---')
                        self.place_buy_order(time=t, price=val, units=self.units)
                        self.position=-1
                    elif self.position==-1 and change<-self.tp:
                        print(50*'-')
                        print('--- TAKE PROFIT (SHORT | -{self.sl}) ---')
                        self.place_sell_order(time=t, price=val, units=self.units)
                        self.position=1
                action=self.get_prediction(data)                
                if self.position==-1 or self.position==0:
                    if action==1:
                        print(50*'_')
                        print(f'{tickval} | --- GO LONG ---')
                        self.place_buy_order(time=t, price=val, amount=self.amount)
                        self.position=1  # long position
                elif self.position==1:
                    if action==-1:
                        print(50*'_')
                        print(f'{tickval} | --- GO SHORT ---')
                        self.place_sell_order(time=t, price=val, units=self.units)
                        self.position=0  # market neutral
            tickval+=1
        self.close_out(time=t, price=val)
        


In [19]:
if __name__ == '__main__':
    test_long=BacktestLongOnly(amount=1000, verbose=True)
    test_long.run_testing()





Running Decision Tree based strategy (online) | lags=5
fixed costs 0.0 | proportional costs 0.0
close 1.17875
close 1.17881
close 1.17866
close 1.17869
close 1.17892
close 1.17891
close 1.1789
close 1.179
close 1.17901
close 1.179
close 1.17895
close 1.1789
close 1.17906
close 1.179
close 1.1791
close 1.17907
close 1.17898
close 1.17904
close 1.17899
close 1.17888
close 1.17888
close 1.17876
close 1.17862
close 1.17852
close 1.17853
close 1.17864
__________________________________________________
25 | --- START TESING ---
--------------------------------------------------
__________________________________________________
25 | --- GO LONG ---
2021-11-12 08:51:07.791438 | buying 848 units at 1.179
2021-11-12 08:51:07.791438 | current balance 0.51
2021-11-12 08:51:07.791438 | current net wealth 1000.00
close 1.17862
__________________________________________________
26 | --- GO SHORT ---
2021-11-12 08:51:09.845599 | selling 848 units at 1.179
2021-11-12 08:51:09.845599 | current balanc

KeyboardInterrupt: 